In [1]:
# Avec Z et le choix des points avec une certaine proba
from deepxrte.geometry import Rectangle
import torch 
import torch.nn as nn 
import torch.optim as optim
from matplotlib.animation import FuncAnimation
from model import PINNs
from utils import read_csv, write_csv
from train import train
from pathlib import Path
import time 
import pandas as pd
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

time_start = time.time()

############# LES VARIABLES ################

folder_result = '4_new_interval'  # le nom du dossier de résultat


torch.manual_seed(42537)

##### Le modèle de résolution de l'équation de la chaleur
nb_itt = 6000      # le nb d'epoch
resample_rate = 3000  # le taux de resampling
display = 500       # le taux d'affichage
poids = [1, 1]   # les poids pour la loss
    
n_data = 5000         # le nb de points initiaux
n_pde = 5000          # le nb de points pour la pde

n_data_test = 5000
n_pde_test  = 5000

Re = 100

lr = 1e-4

x_proba_max = 0.3
x_proba_min = 0.03
y_proba_max = 0.15
y_proba_min = 0.05

proba = 0.8

##### Le code ###############################
###############################################

# La data
df = pd.read_csv('data.csv')


# On adimensionne la data
df_modified = df[(df['Points:0']>= -0.07) & (df['Points:1']>= -0.1) & (df['Points:1']<= 0.1)]
x, y, t = np.array(df_modified['Points:0']), np.array(df_modified['Points:1']), np.array(df_modified['Time'])
u, v, p = np.array(df_modified['Velocity:0']), np.array(df_modified['Velocity:1']), np.array(df_modified['Pressure'])
x = x-x.min()
y = y-y.min()

x_norm = (x-x.mean())/x.std()
y_norm = (y-y.mean())/y.std()
t_norm = (t-t.mean())/t.std()
p_norm = (p-p.mean())/p.std()
u_norm = (u-u.mean())/u.std()
v_norm = (v-v.mean())/v.std()


x_proba_norm_max = (x_proba_max-x.mean())/x.std()
x_proba_norm_min = (x_proba_min-x.mean())/x.std()
y_proba_norm_max = (y_proba_max-y.mean())/y.std()
y_proba_norm_min = (y_proba_min-y.mean())/y.std()


X = np.array([x_norm, y_norm, t_norm], dtype=np.float32).T
U = np.array([u_norm, v_norm, p_norm], dtype=np.float32).T

t_norm_min = t_norm.min()
t_norm_max = t_norm.max()
t_max = t.max()

x_norm_max = x_norm.max()
y_norm_max = y_norm.max()


# On regarde si le dossier existe 
dossier = Path(folder_result)
dossier.mkdir(parents=True, exist_ok=True)

rectangle_proba = Rectangle(x_max = x_proba_norm_max, y_max = y_proba_norm_max,
                      t_min=t_norm_min, t_max=t_norm_max)    # le domaine de plus haute proba 

rectangle = Rectangle(x_max = x_norm_max, y_max = y_norm_max,
                      t_min=t_norm_min, t_max=t_norm_max)    # le domaine de résolution


# les points initiaux du train 
# Les points de pde 


cpu


In [2]:

### Pour train
points_pde_proba = rectangle_proba.generate_random(int(n_pde*proba)).to(device)   # les points pour la pde
points_pde_non_proba = rectangle.generate_random(n_pde-int(n_pde*proba)).to(device)   # les points pour la pde
points_pde = torch.cat((points_pde_proba, points_pde_non_proba), 0)

# On prend des points de data uniquement dans notre rectangle de proba
masque = (X[:,0]> x_proba_min)&(X[:,0]< x_proba_max)&(X[:,1]> y_proba_min)&(X[:,1]< y_proba_max)
points_data_train_proba = np.random.choice(len(X[masque]), int(n_data*proba), replace=False)
inputs_train_data_proba = torch.from_numpy(X[masque][points_data_train_proba]).requires_grad_().to(device)
outputs_train_data_proba = torch.from_numpy(U[masque][points_data_train_proba]).requires_grad_().to(device)
points_data_train_non_proba = np.random.choice(len(X), n_data-int(n_data*proba), replace=False)
inputs_train_data_non_proba = torch.from_numpy(X[points_data_train_non_proba]).requires_grad_().to(device)
outputs_train_data_non_proba = torch.from_numpy(U[points_data_train_non_proba]).requires_grad_().to(device)
inputs_train_data = torch.cat((inputs_train_data_proba, inputs_train_data_non_proba), 0)
outputs_train_data = torch.cat((outputs_train_data_proba, outputs_train_data_non_proba), 0)

### Pour test
X_test_pde = rectangle.generate_random(n_pde_test).to(device)
points_coloc_test = np.random.choice(len(X), n_data_test, replace=False)
X_test_data = torch.from_numpy(X[points_coloc_test]).requires_grad_().to(device)
U_test_data = torch.from_numpy(U[points_coloc_test]).requires_grad_().to(device)


# Initialiser le modèle
model = PINNs().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
loss = nn.MSELoss()

In [20]:
masque = (X[:,0]> x_proba_min)&(X[:,0]< x_proba_max)&(X[:,1]> y_proba_min)&(X[:,1]< y_proba_max)
points_data_train_proba = np.random.choice(len(X[masque]), int(n_data*proba), replace=False)
inputs_train_data_proba = torch.from_numpy(X[masque][points_data_train_proba]).requires_grad_().to(device)
outputs_train_data_proba = torch.from_numpy(U[masque][points_data_train_proba]).requires_grad_().to(device)
points_data_train_non_proba = np.random.choice(len(X), n_data-int(n_data*proba), replace=False)
inputs_train_data_non_proba = torch.from_numpy(X[points_data_train_non_proba]).requires_grad_().to(device)
outputs_train_data_non_proba = torch.from_numpy(U[points_data_train_non_proba]).requires_grad_().to(device)
inputs_train_data = torch.cat((inputs_train_data_proba, inputs_train_data_non_proba), 0)
outputs_train_data = torch.cat((outputs_train_data_proba, outputs_train_data_non_proba), 0)

In [36]:
points_data_train_proba = np.random.choice(len(X[masque]), int(n_data*proba), replace=False)
inputs_train_data_proba = torch.from_numpy(X[masque][points_data_train_proba]).requires_grad_().to(device)
outputs_train_data_proba = torch.from_numpy(U[masque][points_data_train_proba]).requires_grad_().to(device)
points_data_train_non_proba = np.random.choice(len(X), n_data-int(n_data*proba), replace=False)
inputs_train_data_non_proba = torch.from_numpy(X[points_data_train_non_proba]).requires_grad_().to(device)
outputs_train_data_non_proba = torch.from_numpy(U[points_data_train_non_proba]).requires_grad_().to(device)
inputs_train_data = torch.cat((inputs_train_data_proba, inputs_train_data_non_proba), 0)
outputs_train_data = torch.cat((outputs_train_data_proba, outputs_train_data_non_proba), 0)

In [48]:
X[masque].shape

(12426, 3)

In [39]:
points_data_train_non_proba = np.random.choice(len(X), n_data-int(n_data*proba), replace=False)
inputs_train_data_non_proba = torch.from_numpy(X[points_data_train_non_proba]).requires_grad_().to(device)
outputs_train_data_non_proba = torch.from_numpy(U[points_data_train_non_proba]).requires_grad_().to(device)
inputs_train_data = torch.cat((inputs_train_data_proba, inputs_train_data_non_proba), 0)
outputs_train_data = torch.cat((outputs_train_data_proba, outputs_train_data_non_proba), 0)

In [46]:
inputs_train_data = torch.cat((inputs_train_data_proba, inputs_train_data_non_proba), 0)
outputs_train_data = torch.cat((outputs_train_data_proba, outputs_train_data_non_proba), 0)

In [31]:
X[masque]

0.1491799

In [9]:
points_pde_non_proba.shape

torch.Size([1000, 3])

In [10]:
points_pde_proba.shape

torch.Size([4000, 3])

In [11]:
points_pde = torch.cat((points_pde_proba, points_pde_non_proba), 0)

In [13]:
points_pde.shape

torch.Size([5000, 3])